In [1]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

/home/harsh/Main/CGNLP/archive/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [3]:
data = pd.read_csv('./exec.csv')

In [4]:
def encode_data(row):
    question = row["question"]
    context = row["context"]
    inputs = tokenizer(question, context, padding="max_length", max_length=512, truncation=True, return_tensors="pt")
    return inputs

data["encoded_data"] = data.apply(encode_data, axis=1)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


In [5]:
data["encoded_data"]

0      [input_ids, token_type_ids, attention_mask]
1      [input_ids, token_type_ids, attention_mask]
2      [input_ids, token_type_ids, attention_mask]
3      [input_ids, token_type_ids, attention_mask]
4      [input_ids, token_type_ids, attention_mask]
                          ...                     
819    [input_ids, token_type_ids, attention_mask]
820    [input_ids, token_type_ids, attention_mask]
821    [input_ids, token_type_ids, attention_mask]
822    [input_ids, token_type_ids, attention_mask]
823    [input_ids, token_type_ids, attention_mask]
Name: encoded_data, Length: 824, dtype: object

In [6]:
data["input_ids"] = data["encoded_data"].apply(lambda x: x["input_ids"])
data["attention_mask"] = data["encoded_data"].apply(lambda x: x["attention_mask"])
data["token_type_ids"] = data["encoded_data"].apply(lambda x: x.get("token_type_ids"))

In [7]:
from torch.utils.data import DataLoader, TensorDataset

input_ids = torch.stack(data["input_ids"].tolist())
attention_mask = torch.stack(data["attention_mask"].tolist())
token_type_ids = torch.stack(data["token_type_ids"].tolist())

dataset = TensorDataset(input_ids, attention_mask, token_type_ids)

dataloader = DataLoader(dataset, batch_size=16)


In [8]:
model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
import torch

def answer_question(question, context, model, tokenizer):
    inputs = tokenizer(question, context, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits
    
    start_idx = torch.argmax(start_logits)
    end_idx = torch.argmax(end_logits)
    
    answer = tokenizer.decode(inputs.input_ids[0][start_idx:end_idx+1])
    
    return answer

In [10]:
sample_question = "what did the mob do"
sample_context= "mob did not keep peace"
answer = answer_question(sample_question, sample_context, model, tokenizer)
print("Answer:", answer)

Answer: not keep peace


In [11]:
test_data = pd.read_csv('test.csv')

In [12]:
test_data.head()

,Unnamed: 0,id,question,context,answers,context_id
0,0,56be4db0acb8001400a502ec,Which NFL team represented the AFC at Super Bo...,Super Bowl 50 was an American football game to...,"[{'answer_start': 177, 'text': 'Denver Broncos...",0
1,1,56be4db0acb8001400a502ed,Which NFL team represented the NFC at Super Bo...,Super Bowl 50 was an American football game to...,"[{'answer_start': 249, 'text': 'Carolina Panth...",0
2,2,56be4db0acb8001400a502ee,Where did Super Bowl 50 take place?,Super Bowl 50 was an American football game to...,"[{'answer_start': 403, 'text': 'Santa Clara, C...",0
3,3,56be4db0acb8001400a502ef,Which NFL team won Super Bowl 50?,Super Bowl 50 was an American football game to...,"[{'answer_start': 177, 'text': 'Denver Broncos...",0
4,4,56be4db0acb8001400a502f0,What color was used to emphasize the 50th anni...,Super Bowl 50 was an American football game to...,"[{'answer_start': 488, 'text': 'gold'}, {'answ...",0


In [13]:
print("question: ")
print(test_data['question'][3])
print("\ncontext: ")
print(test_data['context'][3])
answer = answer_question(test_data['question'][3], test_data['context'][3], model, tokenizer)
print("\nanswer: ")
print(answer)
print("\noriginal answers: ")
print(test_data['answers'][3])

question: 
Which NFL team won Super Bowl 50?

context: 
Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.

answer: 
denver broncos

original answers: 
[{'answer_start': 177, 'text': 'Denver Broncos'}, {'answer_start': 177, 'text': 'Denver Broncos'}, {'answer_start': 177, 't

In [14]:
test_data.shape

(10570, 6)

In [15]:
for i in range(0, 2067):
    test_data['answers'][i] = [answer.lower() for answer in test_data['answers'][i]]
    test_data['answers'][i] = ''.join(test_data['answers'][i]).lower()
    test_data['answers'][i] = eval(test_data['answers'][i])

/tmp/ipykernel_8334/1439187426.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['answers'][i] = [answer.lower() for answer in test_data['answers'][i]]
/tmp/ipykernel_8334/1439187426.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['answers'][i] = ''.join(test_data['answers'][i]).lower()
/tmp/ipykernel_8334/1439187426.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['answers'][i] = eval(test_data['

In [16]:
test_data['answers'][3]

[{'answer_start': 177, 'text': 'denver broncos'},
 {'answer_start': 177, 'text': 'denver broncos'},
 {'answer_start': 177, 'text': 'denver broncos'}]

In [17]:
test_data.shape

(10570, 6)

In [18]:
test_data.columns

Index(['Unnamed: 0', 'id', 'question', 'context', 'answers', 'context_id'], dtype='object')

In [19]:
test_data = test_data.drop(columns='Unnamed: 0')
test_data = test_data.drop(columns='context_id')
test_data.shape

(10570, 4)

In [20]:
type(test_data['answers'][0])

list

In [21]:
test_data.shape

(10570, 4)

In [22]:
test_data.head()

,id,question,context,answers
0,56be4db0acb8001400a502ec,Which NFL team represented the AFC at Super Bo...,Super Bowl 50 was an American football game to...,"[{'answer_start': 177, 'text': 'denver broncos..."
1,56be4db0acb8001400a502ed,Which NFL team represented the NFC at Super Bo...,Super Bowl 50 was an American football game to...,"[{'answer_start': 249, 'text': 'carolina panth..."
2,56be4db0acb8001400a502ee,Where did Super Bowl 50 take place?,Super Bowl 50 was an American football game to...,"[{'answer_start': 403, 'text': 'santa clara, c..."
3,56be4db0acb8001400a502ef,Which NFL team won Super Bowl 50?,Super Bowl 50 was an American football game to...,"[{'answer_start': 177, 'text': 'denver broncos..."
4,56be4db0acb8001400a502f0,What color was used to emphasize the 50th anni...,Super Bowl 50 was an American football game to...,"[{'answer_start': 488, 'text': 'gold'}, {'answ..."


In [23]:
test_data.shape

(10570, 4)

In [24]:
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

In [25]:
predictions = []
for i in range(0,23):
    predictions.append(answer_question(test_data['question'][i], test_data['context'][i], model, tokenizer))

In [26]:
predictions

['denver broncos',
 'carolina panthers',
 "levi's stadium in the san francisco bay area at santa clara, california",
 'denver broncos',
 'gold',
 'golden anniversary',
 'february 7, 2016',
 'american football conference',
 'golden anniversary',
 'american football conference',
 'february 7, 2016',
 'denver broncos',
 "levi's stadium",
 'santa clara, california',
 'super bowl l',
 '2015',
 '2016',
 'santa clara, california',
 "levi's stadium",
 '24 – 10',
 'february 7, 2016',
 '',
 'denver broncos']

In [27]:
true_answers = test_data['answers'].iloc[0:23]

In [28]:
import nltk
import editdistance

wer_scores = []
em_scores = []

for i in range(len(predictions)):
    prediction = predictions[i]
    answers = true_answers[i]
    
    # Calculate WER score
    min_wer = float('inf')
    max_em = 0.0
    
    for answer in answers:
        wer_score = editdistance.eval(answer.split(), prediction.split())
        min_wer = min(min_wer, wer_score)
        
        if answer == prediction:
            em_score = 1
        else:
            em_score = 0
        
        max_em = max(max_em, em_score)
    
    em_scores.append(max_em)
    wer_scores.append(min_wer)

# Calculate average WER and EM scores
avg_wer = sum(wer_scores) / len(wer_scores)
avg_em = sum(em_scores) / len(em_scores)

print(f"WER Score: {avg_wer:.4f}")
print(f"EM Score: {avg_em:.4f}")


ModuleNotFoundError: No module named 'editdistance'